📄 Script Python : hackernews_fetcher.py

In [ ]:
import requests
import json
import argparse
from pathlib import Path

# URLs de l'API HackerNews
TOP_STORIES_URL = "https://hacker-news.firebaseio.com/v0/topstories.json"
ITEM_URL = "https://hacker-news.firebaseio.com/v0/item/{}.json"

# Dossier de stockage
RAW_DATA_DIR = Path("raw_data")
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)

def get_top_story_ids(limit):
    """
    Récupère les IDs des meilleures actualités.
    """
    response = requests.get(TOP_STORIES_URL)
    if response.status_code != 200:
        raise Exception(f"Erreur API: {response.status_code}")
    story_ids = response.json()
    return story_ids[:limit]

def get_story_details(story_id):
    """
    Récupère les détails d'une actualité.
    """
    response = requests.get(ITEM_URL.format(story_id))
    if response.status_code != 200:
        print(f"⚠️ Impossible de récupérer l'ID {story_id}")
        return None
    return response.json()

def save_to_file(data, filename):
    """
    Sauvegarde les données JSON dans un fichier.
    """
    file_path = RAW_DATA_DIR / filename
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
    print(f"✅ Données sauvegardées dans {file_path}")

def main():
    """
    Point d'entrée principal.
    """
    parser = argparse.ArgumentParser(description="Récupère les dernières actualités de HackerNews.")
    parser.add_argument("--limit", type=int, default=50, help="Nombre d'actualités à récupérer (défaut: 50)")
    args = parser.parse_args()
    
    print("🚀 Récupération des dernières actualités de HackerNews...")
    
    # Étape 1 : Récupérer les IDs
    story_ids = get_top_story_ids(args.limit)
    print(f"📰 {len(story_ids)} IDs récupérés.")
    
    # Étape 2 : Récupérer les détails des actualités
    stories = []
    for i, story_id in enumerate(story_ids, start=1):
        print(f"🔄 Récupération de l'article {i}/{len(story_ids)} (ID: {story_id})")
        story_details = get_story_details(story_id)
        if story_details:
            stories.append(story_details)
    
    # Étape 3 : Sauvegarder les résultats
    save_to_file(stories, "hackernews_raw_data.json")
    print("🎉 Processus terminé avec succès !")

if __name__ == "__main__":
    main()

📌 Code Python pour créer l’index hackernews dans Elasticsearch

In [1]:
import requests
import json

# URL d'Elasticsearch
url = "http://localhost:9200/hackernews"

# En-têtes HTTP
headers = {
    "Content-Type": "application/json"
}

# Corps de la requête (JSON Mapping)
data = {
    "mappings": {
        "properties": {
            "id": {"type": "integer"},
            "title": {"type": "text"},
            "content": {"type": "text"},
            "url": {"type": "keyword"},
            "score": {"type": "integer"},
            "timestamp": {"type": "date"}
        }
    }
}

# Exécution de la requête PUT
response = requests.put(url, headers=headers, data=json.dumps(data))

# Affichage de la réponse
if response.status_code == 200 or response.status_code == 201:
    print("✅ Index 'hackernews' créé avec succès !")
else:
    print(f"❌ Erreur {response.status_code}: {response.text}")

✅ Index 'hackernews' créé avec succès !
